# UNI/ETH Strategy Simulation

In this notebook we will be studying the potential performance of Gamma Strategies' **Strategy 2**, as implemented in ```AutoRegressiveStrategy.py``` on the UNI/ETH pool. This is an interesting pool as these are both "volatile" pairs with some positive correlation, which can help mitigate the impermanent loss relative to a pool that contains a volatile token and a stablecoin.

The strategy uses the AR(1)-GARCH(1,1) model to derive the limits of the liquidity provision ranges, as well as its center based on daily return data. The returns are measured with the change in the relative price of ETH in terms of UNI, which is how the pool is denominated and are the ranges we need to set. It is for this reason that we also need to pull one of the token prices in terms of USD, in order to convert these figures into USD for evaluating the performance of the strategy.

As before, the underlying model is a simple AR(1)-GARCH(1,1) model, which estimates the recent expected return and the volatility of the price based on recent observations.

Suppose that the price at time $t$ of the UNI-WETH 0.3% pool is $q_t$, then the price in USDC terms would be $p_t = 1/q_t$. The return is defined as $r_t = p_t / p_{t-1} - 1$, and this is the time series we model:

$$ r_t = c + \rho r_{t-1} + \sigma_t \varepsilon_t$$
$$ \sigma_t^2 = \omega + \alpha \varepsilon_{t-1} + \beta\sigma_{t-1}^2$$

For the strategy, we estimate the model only with the data the was available at that time, therefore making the simulations more realistic. Note that automatically setting bin width is part of the well-known Bollinger Bands strategy. Imposing a consistent statistical model gives more structure to the optimization problem, as well as allowing for forecasting and simulation more easily.

In [ ]:
import pandas as pd
import numpy as np
import GetPoolData, AutoRegressiveStrategy, ActiveStrategyFramework
import itertools
import arch
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
import scipy
import os


# Create config.py in this directory and enter your own Bitquery API Token
import config
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.GOOGLE_SERVICE_AUTH_JSON

#################
#
# Download data
#
################

# Relevant pool variables - need UNI/ETH and ETH/USDC pools to be able to model USD values 
uni_pool_address      = '0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801'  
eth_usdc_pooL_address = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'
decimals_0            = 18
decimals_1            = 18
decimals_0_eth_usdc   = 6
decimals_1_eth_usdc   = 18

# Download all available data
price_data_begin   = '2020-12-31'
price_data_end     = '2022-12-31'
file_name          = 'eth_uni'
file_name_usd_1    = 'usdc_eth'

uni_pool_data      = GetPoolData.get_pool_data_bigquery(uni_pool_address,price_data_begin,price_data_end,decimals_0,decimals_1)
eth_usdc_pool_data = GetPoolData.get_pool_data_bigquery(eth_usdc_pooL_address,price_data_begin,price_data_end,decimals_0_eth_usdc,decimals_1_eth_usdc)

# ETH USDC price in quotePrice is expressed as ETH / USDC
# Let's generate UNI / USD price as token_0_usd_data
# So need  UNI / ETH ÷ ETH / USDC
eth_usdc_pool_data['quotePrice_eth'] = eth_usdc_pool_data['quotePrice']
token_0_usd_data                     = pd.merge(ActiveStrategyFramework.aggregate_price_data(uni_pool_data,'M'),eth_usdc_pool_data['quotePrice_eth'],left_index=True,right_index=True, how='left').ffill()
token_0_usd_data['quotePrice']       = token_0_usd_data['quotePrice_eth'] / token_0_usd_data['quotePrice']

# Data Analysis

Let's take a look at the UNI/ETH price series and its recent volatility to assess how the strategy will be adjusting ranges over time.

## Price

In [3]:
z_score_cutoff                = 6
window_size                   = 24*30

# Data for strategy simulation cleaning
STRATEGY_FREQUENCY                       = 'H'
# Filter according to Median Absolute Deviation (MAD)
# 1. Generate rolling median
simulate_data_filtered                   = ActiveStrategyFramework.aggregate_price_data(uni_pool_data,STRATEGY_FREQUENCY)
simulate_data_filtered_roll              = simulate_data_filtered.quotePrice.rolling(window=window_size)
simulate_data_filtered['roll_median']    = simulate_data_filtered_roll.median()
roll_dev                                 = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median)
# 2. Compute rolling absolute deviation of current price from median under Gaussian
simulate_data_filtered['median_abs_dev'] = 1.4826*roll_dev.rolling(window=window_size).median()
# 3. Identify outliers based on outliers based on MAD 
outlier_indices                          = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median) >= z_score_cutoff*simulate_data_filtered['median_abs_dev']
simulate_data_filtered                   = simulate_data_filtered[~outlier_indices]
# 4. Generate z-scores
simulate_data_filtered['price_return']   = simulate_data_filtered['quotePrice'].pct_change()

price_chart = pd.DataFrame({'uni-eth': simulate_data_filtered['quotePrice']})
price_chart = price_chart.rename_axis(index={'time_pd':'time'})
price_chart.plot(color='red')

<AxesSubplot:>

## Return

In [4]:
price_chart = pd.DataFrame({'uni-eth return': simulate_data_filtered['price_return']})
price_chart = price_chart.rename_axis(index={'time_pd':'time'})
price_chart.plot(color='red')

<AxesSubplot:>

# Volatility

In [5]:
data_filled_clean        = simulate_data_filtered.dropna(axis=0,subset=['price_return']).copy()

ar_model                 = arch.univariate.ARX(data_filled_clean.price_return, lags=1,rescale=True)
ar_model.volatility      = arch.univariate.GARCH(p=1,q=1)      
res                      = ar_model.fit(update_freq=0, disp="off")

volatility_df            = pd.DataFrame({'uni-eth vol': res.conditional_volatility})
volatility_df['time']    = data_filled_clean.index
volatility_df            = volatility_df.set_index("time")
volatility_df.plot(color='red')

<AxesSubplot:xlabel='time'>

# Simulation

After an optimization process that was conducted to calibrate the relevant parameters, let's look at simulated performance from July 1st to September 10 2021.

In [ ]:
import importlib
importlib.reload(GetPoolData)
importlib.reload(ActiveStrategyFramework)
importlib.reload(AutoRegressiveStrategy)

# Select date ranges for strategy simulation
DATE_BEGIN                 = pd.to_datetime('2021-10-01 00:00PM',utc=True)
DATE_END                   = pd.to_datetime('2022-02-02 00:00PM',utc=True)
z_score_cutoff             = 5
window_size                = 24*30

# Data for strategy simulation. We can use aggregate_price_data to analyze the strategy at a coarser STRATEGY_FREQUENCY in minutes
STRATEGY_FREQUENCY                       = 'H' # evaluate the strategy every minute
simulate_data_filtered                   = ActiveStrategyFramework.aggregate_price_data(uni_pool_data,STRATEGY_FREQUENCY)
simulate_data_filtered_roll              = simulate_data_filtered.quotePrice.rolling(window=window_size) 
simulate_data_filtered['roll_median']    = simulate_data_filtered_roll.median()
roll_dev                                 = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median)
simulate_data_filtered['median_abs_dev'] = 1.4826*roll_dev.rolling(window=window_size).median()
outlier_indices                          = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median) >= z_score_cutoff*simulate_data_filtered['median_abs_dev']
simulate_data_price                      = simulate_data_filtered[~outlier_indices]['quotePrice'][DATE_BEGIN:DATE_END]

import importlib
importlib.reload(ActiveStrategyFramework)

# Data for statistical analaysis (AGGREGATED_MINUTES frequency data)
STAT_MODEL_FREQUENCY       = 'H'

# USD Data remove outliers
token_0_usd_data_filtered  = token_0_usd_data[DATE_BEGIN:DATE_END].copy()
initial_token_0_usd        = token_0_usd_data_filtered['quotePrice'].iloc[0]

# Initial Position Details
initial_usd_price          = 1 / token_0_usd_data_filtered['quotePrice'].iloc[0]
INITIAL_TOKEN_0            = 100000 / initial_usd_price
INITIAL_TOKEN_1            = INITIAL_TOKEN_0*simulate_data_price[0]
FEE_TIER                   = 0.003

# Set decimals according to your pool
DECIMALS_0 = 18
DECIMALS_1 = 18

# Strategy Parameters
alpha_range        = [1.5,1.75,2.0]
tau_range          = [.05,.25,.5]
vol_range          = [.85]
ret_forecast_range = [.5]
sim_performance    = []
sim_results        = []
sim_data_collect   = []

########################################
# Single thread
########################################

for j in list(itertools.product(alpha_range,tau_range,vol_range,ret_forecast_range)):
    print(j)
    AR_Strategy                      = AutoRegressiveStrategy.AutoRegressiveStrategy(uni_pool_data,j[0],j[1],j[2],data_frequency=STAT_MODEL_FREQUENCY,return_forecast_cutoff=j[3],z_score_cutoff=5)
    simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,uni_pool_data,AR_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
    sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,AR_Strategy,token_0_usd_data=token_0_usd_data_filtered)
    strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data,frequency=STRATEGY_FREQUENCY)
    strat_result['alpha_param']      = j[0]
    strat_result['tau_param']        = j[1]
    strat_result['volatility_param'] = j[2]
    sim_results.append(simulated_strategy)
    sim_performance.append(strat_result)
    sim_data_collect.append(sim_data)

########################################
# Parallelized for speed
########################################

# def run_hypervisor(j):
#     AR_Strategy                      = AutoRegressiveStrategy.AutoRegressiveStrategy(uni_pool_data,j[0],j[1],j[2],data_frequency=STAT_MODEL_FREQUENCY,return_forecast_cutoff=.5,z_score_cutoff=5)
#     simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,uni_pool_data,AR_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
#     sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,AR_Strategy,token_0_usd_data=token_0_usd_data_filtered)
#     strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data,frequency=STRATEGY_FREQUENCY)
#     strat_result['alpha_param']      = j[0]
#     strat_result['tau_param']        = j[1]
#     strat_result['volatility_param'] = j[2]
    
#     results = {     
#                 'simulated_strategy' : simulated_strategy,
#                 'strat_result'       : strat_result,
#                 'sim_data'           : sim_data
#     }
    
#     return results

# import multiprocessing
# def multip(file_in):
#     with multiprocessing.Pool(4) as pool:
#             processes = pool.map(run_hypervisor, file_in)
#     return processes

# sim_results = multip(itertools.product(alpha_range,tau_range,vol_range))
# sim_performance = [x['strat_result'] for x in sim_results]


In [15]:
pd.DataFrame(sim_performance)

,days_strategy,gross_fee_apr,gross_fee_return,net_apr,net_return,rebalances,max_drawdown,volatility,sharpe_ratio,impermanent_loss,mean_base_position,median_base_position,mean_base_width,median_base_width,final_value,alpha_param,tau_param,volatility_param
0,124,0.059100,0.020078,-1.040579,-0.353512,66,0.600057,1.044565,-0.996185,-0.001498,0.808863,0.765375,1.831334,1.835841,129297.617926,1.50,0.05,0.85
1,124,0.064886,0.022043,-1.128536,-0.383393,6,0.604086,1.021433,-1.104856,-0.047650,0.642320,0.622316,1.673419,1.697068,123321.359480,1.50,0.25,0.85
2,124,0.071793,0.024390,-1.130716,-0.384134,4,0.604043,1.021368,-1.107060,-0.048793,0.633745,0.667625,1.701346,1.810193,123173.288902,1.50,0.50,0.85
3,124,0.050130,0.017030,-1.030427,-0.350063,49,0.597359,1.041926,-0.988963,0.003829,0.772051,0.735677,2.065618,2.068147,129987.443937,1.75,0.05,0.85
4,124,0.054261,0.018434,-1.107086,-0.376106,8,0.606976,1.033491,-1.071210,-0.036395,0.698000,0.698993,1.968181,1.962420,124778.795305,1.75,0.25,0.85
5,124,0.061450,0.020876,-1.104957,-0.375383,5,0.602687,1.027808,-1.075061,-0.035277,0.654932,0.668315,1.979931,2.071993,124923.483515,1.75,0.50,0.85
6,124,0.046577,0.015823,-1.026350,-0.348678,40,0.596867,1.043804,-0.983279,0.005968,0.788801,0.749618,2.234759,2.223741,130264.427018,2.00,0.05,0.85
7,124,0.046586,0.015826,-1.058454,-0.359585,8,0.597981,1.034147,-1.023505,-0.010877,0.694700,0.673017,2.133644,2.204656,128083.092785,2.00,0.25,0.85
8,124,0.029023,0.009860,-1.108370,-0.376542,3,0.603726,1.027517,-1.078687,-0.037068,0.700425,0.704162,2.055022,2.038280,124691.584820,2.00,0.50,0.85


We can do a bit better, let's run a simple optimizer that is trying to minimize the impermanent loss of the strategy.

In [ ]:
########################################
# Optimizer code
########################################

from scipy import optimize
def run_hypervisor_single(j):
    AR_Strategy                      = AutoRegressiveStrategy.AutoRegressiveStrategy(uni_pool_data,j[0],j[1],j[2],data_frequency=STAT_MODEL_FREQUENCY,return_forecast_cutoff=.5,z_score_cutoff=5)
    simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,uni_pool_data,AR_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
    sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,AR_Strategy,token_0_usd_data=token_0_usd_data_filtered)
    strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data,frequency=STRATEGY_FREQUENCY)
    print('Guess {}: || IL: {}'.format(j,strat_result['impermanent_loss']))
    return -strat_result['impermanent_loss']

o1 = optimize.minimize(fun=run_hypervisor_single, x0=np.array([2.25,.025,.85]),method='COBYLA')

In [18]:
AR_Strategy                      = AutoRegressiveStrategy.AutoRegressiveStrategy(uni_pool_data,o1.x[0],o1.x[1],o1.x[2],data_frequency=STAT_MODEL_FREQUENCY,return_forecast_cutoff=.5,z_score_cutoff=5)
simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,uni_pool_data,AR_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,AR_Strategy,token_0_usd_data=token_0_usd_data_filtered)
strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data,frequency=STRATEGY_FREQUENCY)
print(strat_result)

{'days_strategy': 124, 'gross_fee_apr': 0.043888097829380315, 'gross_fee_return': 0.014909929125597696, 'net_apr': -1.0029628432945175, 'net_return': -0.3407325823795073, 'rebalances': 302, 'max_drawdown': 0.5929057709979699, 'volatility': 1.0441239492427987, 'sharpe_ratio': -0.9605783336565248, 'impermanent_loss': 0.01823951157646392, 'mean_base_position': 0.8166767561322698, 'median_base_position': 0.8063375089765618, 'mean_base_width': 2.4698557760273796, 'median_base_width': 2.42738870259095, 'final_value': 131853.48352409853}


In [ ]:
resulting_data     = sim_data
p = ActiveStrategyFramework.plot_strategy(resulting_data,'ETH / UNI Price')

In [ ]:
p = ActiveStrategyFramework.plot_position_value(resulting_data)

In [ ]:
p = ActiveStrategyFramework.plot_asset_composition(resulting_data,'UNI','WETH')

In [ ]:
p = ActiveStrategyFramework.plot_position_return_decomposition(resulting_data)

In [ ]:
p = ActiveStrategyFramework.plot_position_composition(resulting_data)